In [1]:
#inspiration from here https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
!pip install cmake 'gym[atari]' scipy

    100% |████████████████████████████████| 36.4MB 384kB/s 
    100% |████████████████████████████████| 1.5MB 2.9MB/s 
    100% |████████████████████████████████| 1.0MB 3.5MB/s 
    100% |████████████████████████████████| 3.3MB 4.3MB/s 
    100% |████████████████████████████████| 1.2MB 6.1MB/s 
    100% |████████████████████████████████| 829kB 6.1MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / - done
  Stored in directory: /Users/chloeloughridge/Library/Caches/pip/wheels/57/b0/13/4153e1acab826fbe612c95b1336a63a3fa6416902a8d74a1b7
  Running setup.py bdist_wheel for PyOpenGL ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /Users/chloeloughridge/Library/Caches/pip/wheels/6c/00/7f/1dd736f380848720ad79a1a1de5272e0d3f79c15a42968fb58
  Running setup.py bdist_wheel for future ... - \ | / done
  Stored in directory: /Users/chloeloughridge/Library/Caches/pip/wheels/0c/61/d2/d6b7317325828fbb39ee

In [1]:
import gym

In [2]:
env = gym.make("Taxi-v2").env
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
# navigating the environment randomly first
env.s = 328
epochs = 0
penalties, reward = 0, 0
frames = []
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action':action,
            'reward': reward
        })
    epochs += 1
    
print("timesteps taken: {}".format(epochs))
print("penalties incurred: {}".format(penalties))
print("rewards: {}".format(reward))

timesteps taken: 864
penalties incurred: 285
rewards: 20


In [11]:
#animation!
# THERE IS SOMETHING GOING WRONG HERE
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print("Timestep: {i + 1}")
        print("State: {frame['state']}")
        print("Action: {frame['action']}")
        print("Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

AttributeError: 'str' object has no attribute 'getvalue'